#### Dashboard for investments

Idea: We want to invest when there is significant deviation away from the long-term mean

Steps:
1. Calculate long term trend of SNP500 - Loop through each time period in the dataframe and then conduct a ols regression and store the outputs. The bins are 15 year periods (12 * 15)
2. Determine at which points was there significant deviation away from trend (Jacksobian co-integration test)
3. Create threshold at which point there was significant deviation away that led you to an investment decision 
4. Create algorithm that invests when this happens

Data: 
1. Currently we just have a randomly CSV of the data, our goal is to find a constant datasource.

In [1]:
import os
import pandas as pd 
import numpy as np 
import matplotlib as plt 
from datetime import datetime

In [2]:
path = os.getcwd()
path

'/Users/dylanjohnson/Documents/bos_investments/bos_investments'

In [3]:
env = os.chdir("./Documents/bos_investments")

FileNotFoundError: [Errno 2] No such file or directory: './Documents/bos_investments'

In [5]:
df = pd.read_csv("s&p500.csv", infer_datetime_format = True)
df.head(6)

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,1871-01-01,4.44,0.26,0.4,12.46,5.32,89.00,5.21,8.02,NaN
1,1871-02-01,4.50,0.26,0.4,12.84,5.32,87.53,5.06,7.78,NaN
2,1871-03-01,4.61,0.26,0.4,13.03,5.33,88.36,4.98,7.67,NaN
3,1871-04-01,4.74,0.26,0.4,12.56,5.33,94.29,5.17,7.96,NaN
4,1871-05-01,4.86,0.26,0.4,12.27,5.33,98.93,5.29,8.14,NaN
5,1871-06-01,4.82,0.26,0.4,12.08,5.34,99.66,5.38,8.27,NaN


### Step 1: Create the bins

What do we want to do:
1. Create a bin for each time_stamp

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [37]:
df

,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,1871-01-01,4.44,0.26,0.40,12.46,5.32,89.00,5.21,8.02,NaN
1,1871-02-01,4.50,0.26,0.40,12.84,5.32,87.53,5.06,7.78,NaN
2,1871-03-01,4.61,0.26,0.40,13.03,5.33,88.36,4.98,7.67,NaN
3,1871-04-01,4.74,0.26,0.40,12.56,5.33,94.29,5.17,7.96,NaN
4,1871-05-01,4.86,0.26,0.40,12.27,5.33,98.93,5.29,8.14,NaN
...,...,...,...,...,...,...,...,...,...,...
1763,2017-01-12,2664.34,48.93,109.88,246.52,2.40,2700.13,49.59,111.36,32.09
1764,2018-01-01,2789.80,49.29,NaN,247.87,2.58,2811.96,49.68,NaN,33.31
1765,2018-01-02,2705.16,49.64,NaN,248.99,2.86,2714.34,49.81,NaN,32.12
1766,2018-01-03,2702.77,50.00,NaN,249.55,2.84,2705.82,50.06,NaN,31.99


In [9]:
a = pd.to_datetime(df['Date'])

Make a new column that tells the bins that it has

In [61]:
print(len(a))
rows = 1769
print(rows)
bin_size = 15*12
print(bin_size)

1768
1769
180


In [13]:
col_names = []
for i in range(len(a)):
    col_names.append("bin_"+str(i)) 

In [14]:
col_names

['bin_0',
 'bin_1',
 'bin_2',
 'bin_3',
 'bin_4',
 'bin_5',
 'bin_6',
 'bin_7',
 'bin_8',
 'bin_9',
 'bin_10',
 'bin_11',
 'bin_12',
 'bin_13',
 'bin_14',
 'bin_15',
 'bin_16',
 'bin_17',
 'bin_18',
 'bin_19',
 'bin_20',
 'bin_21',
 'bin_22',
 'bin_23',
 'bin_24',
 'bin_25',
 'bin_26',
 'bin_27',
 'bin_28',
 'bin_29',
 'bin_30',
 'bin_31',
 'bin_32',
 'bin_33',
 'bin_34',
 'bin_35',
 'bin_36',
 'bin_37',
 'bin_38',
 'bin_39',
 'bin_40',
 'bin_41',
 'bin_42',
 'bin_43',
 'bin_44',
 'bin_45',
 'bin_46',
 'bin_47',
 'bin_48',
 'bin_49',
 'bin_50',
 'bin_51',
 'bin_52',
 'bin_53',
 'bin_54',
 'bin_55',
 'bin_56',
 'bin_57',
 'bin_58',
 'bin_59',
 'bin_60',
 'bin_61',
 'bin_62',
 'bin_63',
 'bin_64',
 'bin_65',
 'bin_66',
 'bin_67',
 'bin_68',
 'bin_69',
 'bin_70',
 'bin_71',
 'bin_72',
 'bin_73',
 'bin_74',
 'bin_75',
 'bin_76',
 'bin_77',
 'bin_78',
 'bin_79',
 'bin_80',
 'bin_81',
 'bin_82',
 'bin_83',
 'bin_84',
 'bin_85',
 'bin_86',
 'bin_87',
 'bin_88',
 'bin_89',
 'bin_90',
 'bin_91'

In [15]:
bins = pd.DataFrame(columns = col_names)
bins = bins.reindex(list(range(0, 1770))).reset_index(drop=True)
bins

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,...,bin_1758,bin_1759,bin_1760,bin_1761,bin_1762,bin_1763,bin_1764,bin_1765,bin_1766,bin_1767


In [29]:
n = rows
lists = [[] for _ in range(n)]
lists
bins.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'bin_5', 'bin_6', 'bin_7',
       'bin_8', 'bin_9',
       ...
       'bin_1758', 'bin_1759', 'bin_1760', 'bin_1761', 'bin_1762', 'bin_1763',
       'bin_1764', 'bin_1765', 'bin_1766', 'bin_1767'],
      dtype='object', length=1768)

In [31]:
columns = len(bins.columns)

In [55]:
print(len(bins))
print(len(bins.columns))
print(bin_size)

1770
1768
180


In [56]:
def create_empty_lists():    
    n = 1768
    lists = [[] for _ in range(n)]
    return lists
lists = create_empty_lists()

In [57]:
def bin_defined(columns, rows, bin_size, empty_list):
    for i in range(len(columns)): ## For each bin (column)
        for y in range(rows): #for each row
            if i <= y <= i+bin_size: ##for each index within our bin our length assign the index to the correct bin 
                lists[i].append(1)
            else:
                lists[i].append(0)
    return lists
l_of_l = bin_defined(columns = bins.columns, rows = len(bins), bin_size = 180, empty_list = lists)

In [58]:
def convert_dataframe(l_of_l):
    col_names = []
    
    df = pd.DataFrame(l_of_l)
    df = df.iloc[:,:-2]
    df = df.transpose()

    
    for i in range(1768):
        col_names.append("bin_" + str(i))
    df.columns = [col_names]
    
    return df 
df = convert_dataframe(l_of_l)

### Step 2: Map these bins to the data that we have

### Step 3: Do an OLS regression for each bin